## Server setup

We setup a standard NanoVer server with one of the test files.

In [1]:
from nanover.omni import OmniRunner
from nanover.omni.openmm import OpenMMSimulation

simulation = OpenMMSimulation.from_xml_path("../openmm/openmm_files/17-ala.xml")
runner = OmniRunner.with_basic_server(simulation)

runner.load(0)
runner.print_basic_info()

Serving "ragzo: NanoVer iMD Server" (ws://localhost:62511), discoverable on all interfaces on port 54545
Available simulations:
0: "17-ala"


With the server running, we just need the connection address:

In [2]:
from nanover.websocket.client.app_client import get_websocket_address_from_app_server

ADDRESS = get_websocket_address_from_app_server(runner.app_server)
ADDRESS

'ws://localhost:62511'

## Connecting without NanoVer

It is fairly straight-forward to access a NanoVer server using just widely available libraries.

Using a standard WebSocket implemention, we can connect to the address from earlier and directly receive data:

In [3]:
from websockets.asyncio.client import connect

async with connect(ADDRESS) as websocket:
    print(await websocket.recv())
    print(await websocket.recv())

b'\x81\xa5frame\xde\x00\x18\xabframe.index\x00\xb9system.simulation.counter\x00\xb2particle.positions\xc5\x08\x1cB\xc2\x8a\xbe\xa15\t=\xe5&\x1eA\x90\x9a\x0b\xbeKM3\xbb\xce_\x1dAf\x18\xe0\xbc\x99(\xa0=?\x84\x1eA\'\x1d\x81\xbd\x19>>>1`\x1fA\ry\xff\xbdL\x1a\xec;\xa4\xed\x1aA1\xc3\xaa\xbe\xbc\x9a\x12\xbd}\xa5\x1dA\xb2\'\x8c\xbe\xe2^\xef<\x9b\xce\x1fA\x0b@\x9a\xbe\xb7\xdb\xff=\xfc\xd1\x1dA\tE\xfd\xbdL!\xd5\xbd\x9b\xc5\x1dA\x1b\x15R\xbe\xe5\x88O\xbd\x8c\r\x1aAh\x04\x13\xbe5\x88\xda=mh\x1aA&Z\xef\xbca\xec\xc6\xbc\xa4j\x1aA\xb3\\\xc8=\x11\x0b\xf3<\x0f\x86\x1eA\xe9\x87\\>h\xab\xc6=\xbct\x1fA\xa7p\x9b>\xaa\x9d->\x9b\xd0\x1dAq\xfd\x91>\x8e\xff\x18>\xbb\xed\x1bA\x0ew\x92>f\xb5\x0c\xba\x0e\xdb A\x10\xbf\xf4=\xdc\x08y\xbd\xe4\x0c\x1eAb#:>\x87\x825>\xad\x8e A\x87\xf6Q>)\xc8\xa7\xbcF\xfb!AdC\xa2>v\xde\xb2\xbdZ\x07 A\x85A\xbe>\x0e\x92P=\xe9\xb7!A\xa2\xbf\xca>\xc3>{>\xeb\x9c\x1eA\x17\x13\x00?VW\xa0>\x06Z\x1dA\xcf\xb1#?$\x0f\x9a>\xbcZ\x1eA"r\'?>\xfe\x97>\x03R A\xdff\xe9>\x86\xc7\xeb>\xca\x13\x1dA\xa1\xb9

The data is raw bytes, and not human-readable, but it is in the widely supported MessagePack format, which we can decode easily:

In [4]:
import msgpack
import pprint

async with connect(ADDRESS) as websocket:
    data = await websocket.recv()
    message = msgpack.unpackb(data)
    pprint.pp(message)
    data = await websocket.recv()
    message = msgpack.unpackb(data)
    pprint.pp(message)

{'state': {'updates': {'imd.velocity_reset_available': False}, 'removals': []}}
{'frame': {'frame.index': 0,
           'system.simulation.counter': 0,
           'particle.positions': b'\xf7\xfdv\xbe\x17\xa2\\\xbd\x9a4\x1dA'
                                 b'\xf0E\x02\xbe\x9dZ\xb1\xbc\x80\xc1\x1bA'
                                 b'cQ\x89\xbb\xdc\x13n\xbb\xd1(\x1dA'
                                 b'\xa0l\xa2\xbc\x0c-\x8c<\x98\x17\x1fAv9(\xbe'
                                 b'j3\xe0=8o\x1aAw\x00\xa6\xbej\x17\x84\xbb'
                                 b'\xa0\xc2\x1cA\xb1\xc8\x83\xbe\xec[\x17\xbe'
                                 b'\x90\x97\x1dA\xf1\x82X\xbe@\x0c\x7f\xbc'
                                 b'\xe2\xaf\x1eAE3\xde\xbd_\x98\xd9\xbd'
                                 b'!\xc3\x1aA\x08qo\xbe@G\xb5=(\xff\x18A'
                                 b'\x94;Z\xbe\x14\xc2=>\xa0b\x1bAqQ\x9a\xbd'
                                 b'\xf3\x11&>\xb1\x02\x1aAmG\xeb=\xee\x1ch:'
               

Each message is a dictionary where each key is a message type and the value is the message of that type. We'll focus on the simulation and find the first message that has type "frame":

In [5]:
async with connect(ADDRESS) as websocket:
    async for data in websocket:
        message = msgpack.unpackb(data)
        if "frame" in message:
            FIRST_FRAME = message["frame"]
            break

pprint.pp(FIRST_FRAME)

{'frame.index': 0,
 'system.simulation.counter': 0,
 'particle.positions': b';\x93\x80\xbe\x94\x83\x9f\xbb\xc5~\x1aA9K\x02\xbe'
                       b'\xf3\x9f7\xbbAG\x19A\n\xe6\x89\xbbn\x99\x8f\xbc'
                       b'\x18\xbe\x1aA\xb9\x90\xd4\xbc\xf7\xe7$\xbd'
                       b'\xa3\xb1\x1cA=@\xe9\xbd[5\n>\xf5\x18\x18A'
                       b'\xe9\xdb\x81\xbe\x92LW=|\xbc\x1bAD\x9e\xab\xbe'
                       b'\xe8\x06\x9e<\xf2\xac\x19A\x10}\x85\xbe'
                       b'\xe1\x19\xcf\xbdD\xe4\x1aA\xabt\t\xbe\x98\xea\xb7\xbd'
                       b'\x05+\x18A\x9a\xeaV\xbe\xe6\xdf?>G\xa5\x17A\x15F~\xbd'
                       b'@\xf7O>W!\x19A\xaa\xce\x80\xbd3\x02\xfa=\xf2\xa5\x16A'
                       b'8\xbd\xfc=\x0ff\xcb\xbc:\x02\x1aA\\\x1a~>k\x97U\xbd'
                       b'\x9d3\x1bA\x80\xdd\xbf>\x96\x1f[\xbc\xab\xf3\x19A'
                       b'\xaf#\xbd>M\xe1\x08\xbd~\r\x18A\xe5\x9fq>0#V\xbe'
                       b'\xe2\xd6\x1bA\x83\xb5\x07>1\

In general the frame messages provide continuous updates about the values of the simulated system as they change. The first frame message contains the entire initial state. The message is largely human-readable, but the positions are still raw bytes:

In [6]:
POSITION_DATA = FIRST_FRAME["particle.positions"]
POSITION_DATA

b';\x93\x80\xbe\x94\x83\x9f\xbb\xc5~\x1aA9K\x02\xbe\xf3\x9f7\xbbAG\x19A\n\xe6\x89\xbbn\x99\x8f\xbc\x18\xbe\x1aA\xb9\x90\xd4\xbc\xf7\xe7$\xbd\xa3\xb1\x1cA=@\xe9\xbd[5\n>\xf5\x18\x18A\xe9\xdb\x81\xbe\x92LW=|\xbc\x1bAD\x9e\xab\xbe\xe8\x06\x9e<\xf2\xac\x19A\x10}\x85\xbe\xe1\x19\xcf\xbdD\xe4\x1aA\xabt\t\xbe\x98\xea\xb7\xbd\x05+\x18A\x9a\xeaV\xbe\xe6\xdf?>G\xa5\x17A\x15F~\xbd@\xf7O>W!\x19A\xaa\xce\x80\xbd3\x02\xfa=\xf2\xa5\x16A8\xbd\xfc=\x0ff\xcb\xbc:\x02\x1aA\\\x1a~>k\x97U\xbd\x9d3\x1bA\x80\xdd\xbf>\x96\x1f[\xbc\xab\xf3\x19A\xaf#\xbd>M\xe1\x08\xbd~\r\x18A\xe5\x9fq>0#V\xbe\xe2\xd6\x1bA\x83\xb5\x07>1\xee\xa6\xbc\xd3n\x18A\xcfp}>\xc3:\xc7;o\xb3\x1cA\x1f\xc2\x0e>\x1a\x99l\xbe\xcb\x98\x1cAw\xc1|>\xbdP\x89\xbeD`\x1aA\xd8\x8e\xa3>\xc8zz\xber\xcb\x1cA \x96\xf4>\x7f\xda!=\x11\x08\x1bA\x925\x15?\xdb\xf6\xf2=\xe6\xee\x19A\x1f:3?\x0f\xdc\x14>5w\x1bA\xd3F/?\x0f\xf0\x03>"]\x1dA\xf7\x81\x05?\xe8\xe9\x83>\x91]\x19A`\xbb\xf4>\x93\xee3=\x86\xa6\x1cA\xd5f ?\xdd\xfc\x97=b\x82\x18A?\x93\xd6>D\x96|>^\xb4\x18A\xf

The positions are sent as an array of 32 bit floating point numbers, with three numbers per position, and 4 bytes per number, this 173 atom system should have 2076 bytes of positions data:

In [7]:
print(len(POSITION_DATA), len(POSITION_DATA) // 12)

2076 173


We can use numpy to easily transform these bytes into the 2d array of positions (and in other languages and environments there are tools to do it just as easily):

In [8]:
import numpy as np

POSITIONS = np.frombuffer(POSITION_DATA, dtype=np.float32)
POSITIONS.reshape(-1, 3)

array([[-2.51123279e-01, -4.86798026e-03,  9.65594959e+00],
       [-1.27240077e-01, -2.80189211e-03,  9.57989597e+00],
       [-4.20833100e-03, -1.75292157e-02,  9.67140961e+00],
       [-2.59479154e-02, -4.02602814e-02,  9.79336834e+00],
       [-1.13892056e-01,  1.34969160e-01,  9.50609303e+00],
       [-2.53630906e-01,  5.25632575e-02,  9.73351669e+00],
       [-3.35191846e-01,  1.92904025e-02,  9.60472298e+00],
       [-2.60719776e-01, -1.01123579e-01,  9.68072891e+00],
       [-1.34234115e-01, -8.98029208e-02,  9.51050282e+00],
       [-2.09879309e-01,  1.87377542e-01,  9.47785091e+00],
       [-6.20785542e-02,  2.03091621e-01,  9.57063961e+00],
       [-6.28941804e-02,  1.22074507e-01,  9.41551399e+00],
       [ 1.23407781e-01, -2.48289388e-02,  9.62554359e+00],
       [ 2.48147428e-01, -5.21463566e-02,  9.70010090e+00],
       [ 3.74736786e-01, -1.33742299e-02,  9.62198925e+00],
       [ 3.69412869e-01, -3.34179886e-02,  9.50329399e+00],
       [ 2.35961512e-01, -2.09118605e-01